# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import pathlib

output_data_file = "output_data/cities.csv"

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = pathlib.Path(output_data_file)
original_df = pd.read_csv(filepath)
original_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,talara,-4.58,-81.27,296.32,79,16,7.11,PE,1588978140
1,hithadhoo,-0.60,73.08,302.45,78,84,7.55,MV,1588978184
2,hobyo,5.35,48.53,302.12,75,0,4.31,SO,1588978184
3,provideniya,64.38,-173.30,278.96,65,97,7.18,RU,1588978185
4,simplicio mendes,-7.85,-41.91,296.77,90,100,1.66,BR,1588978185


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
weather_df = original_df.copy()
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[['Lat', 'Lng']].astype(float)
locations.dropna()
humidity_num = weather_df['Humidity'].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

# humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_num)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_num, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:
max_temp = (80 + 459.67)*5/9
min_temp = (70 + 459.67)*5/9
max_wind_speed = 10
cloudiness = 0

temp_boolean = ((weather_df['Max Temp'] >= min_temp) & (weather_df['Max Temp'] < max_temp))
wind_speed_boolean = (weather_df['Wind Speed'] <= max_wind_speed)
cloudiness_boolean = (weather_df['Cloudiness'] == cloudiness)

weather_filtered_df = weather_df.loc[(temp_boolean & wind_speed_boolean & cloudiness_boolean),:]
weather_filtered_df = weather_filtered_df.reset_index(drop=True)
weather_filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,turayf,31.67,38.66,297.15,29,0,8.20,SA,1588978250
1,tura,25.52,90.22,294.97,44,0,1.49,IN,1588978289
2,minab,27.15,57.08,298.15,78,0,2.10,IR,1588978306
3,santa cruz,-17.80,-63.17,295.15,43,0,2.10,BO,1588978333
4,akyab,20.15,92.90,299.35,70,0,1.88,MM,1588978291
5,poum,-20.23,164.02,296.67,67,0,1.69,NC,1588978340
6,walvis bay,-22.96,14.51,295.57,39,0,2.15,NaN,1588978225
7,sao filipe,14.90,-24.50,296.54,70,0,5.95,CV,1588978376
8,robertsganj,24.70,83.07,299.28,25,0,2.53,IN,1588978385
9,asyut,27.18,31.18,297.15,44,0,8.20,EG,1588978254


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_df = pd.DataFrame()
target_search = 'Hotel'

lat = weather_filtered_df['Lat']
lng = weather_filtered_df['Lng']

for i in range(len(weather_filtered_df['City'])):
    # redefine params
    params = {
        "location": f'{lat[i]},{lng[i]}',
        "keyword": target_search,
        "radius": 50000,
        "key": g_key
    }
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params).json()
    hotel_df = hotel_df.append([[
        hotel['results'][0]['name'],
        hotel['results'][0]['geometry']['location']['lat'],
        hotel['results'][0]['geometry']['location']['lng'],
        weather_filtered_df['City'][i],
        weather_filtered_df['Country'][i]
    ]])

In [64]:
hotel_df.columns=['Hotel Name', 'Lat', 'Lng', 'City', 'Country']
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Golden Dune Hotel,31.672446,38.659673,turayf,SA
1,Hotel Polo Orchid,25.514468,90.192206,tura,IN
2,مجتمع صددف,27.135314,57.081756,minab,IR
3,Los Tajibos Hotel & Convention Center,-17.764532,-63.196282,santa cruz,BO
4,Nawarat Hotel,20.595298,93.193760,akyab,MM
5,Hotel Karem Bay,-20.547754,164.274260,poum,NC
6,Strand Hotel Swakopmund,-22.675519,14.521949,walvis bay,NaN
7,Hotel Ocean View & Restaurante Seafood,14.892671,-24.500622,sao filipe,CV
8,hotel satyam shivam sundaram,24.477712,83.031835,robertsganj,IN
9,Florence Hotel Assiut,27.187841,31.193165,asyut,EG


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [70]:
# Create a combined map
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))